<a href="https://colab.research.google.com/github/Blusooyeon/Studying_for_Bigdata_Test/blob/main/T2_3_adult_income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 성인 인구조사 소득 예측

In [43]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name

    if null_name != "":
        df[df == null_name] = np.nan

    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)

    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test

df = pd.read_csv("/content/adult.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='income', null_name='?')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26048, 15), (6513, 15), (26048, 2), (6513, 2))

### Start

In [44]:
import pandas as pd
import numpy as np

In [45]:
X_train.shape , X_test.shape , y_train.shape

((26048, 15), (6513, 15), (26048, 2))

In [46]:
X_train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
21851,21851,36,Private,241998,Bachelors,13,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,50,United-States
7632,7632,53,Private,103950,Masters,14,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States
27878,27878,19,Private,203061,Some-college,10,Never-married,Tech-support,Not-in-family,White,Female,0,0,25,United-States
14121,14121,20,Private,102607,HS-grad,9,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,30,United-States
32345,32345,54,State-gov,138852,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States


In [47]:
# 타겟 수 확인
y_train['income'].value_counts()

<=50K    19756
>50K      6292
Name: income, dtype: int64

In [48]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26048 entries, 21851 to 25716
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              26048 non-null  int64 
 1   age             26048 non-null  int64 
 2   workclass       24592 non-null  object
 3   fnlwgt          26048 non-null  int64 
 4   education       26048 non-null  object
 5   education.num   26048 non-null  int64 
 6   marital.status  26048 non-null  object
 7   occupation      24585 non-null  object
 8   relationship    26048 non-null  object
 9   race            26048 non-null  object
 10  sex             26048 non-null  object
 11  capital.gain    26048 non-null  int64 
 12  capital.loss    26048 non-null  int64 
 13  hours.per.week  26048 non-null  int64 
 14  native.country  25587 non-null  object
dtypes: int64(7), object(8)
memory usage: 3.2+ MB


In [49]:
# feature 구분
numeric_features = [
                    'age',
                    'fnlwgt',
                    'education.num',
                    'capital.loss',
                    'capital.gain',
                    'hours.per.week'
                  ]

cat_features = [
                 'workclass',
                 'education',
                 'marital.status',
                 'occupation',
                 'relationship',
                 'race',
                 'sex',
                 'native.country'
                 ]


In [50]:
X_train[numeric_features].describe()

,age,fnlwgt,education.num,capital.loss,capital.gain,hours.per.week
count,26048.000000,2.604800e+04,26048.000000,26048.000000,26048.000000,26048.000000
mean,38.610335,1.895741e+05,10.082118,88.477695,1081.193796,40.420224
std,13.628346,1.043848e+05,2.574608,404.689981,7404.962675,12.354707
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.182472e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.785755e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.365968e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,4356.000000,99999.000000,99.000000


In [51]:
X_train[cat_features].describe()

,workclass,education,marital.status,occupation,relationship,race,sex,native.country
count,24592,26048,26048,24585,26048,26048,26048,25587
unique,8,16,7,14,6,5,2,41
top,Private,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
freq,18160,8408,11987,3323,10558,22270,17400,23381


### 결측치 처리

In [52]:
X_train.isnull().sum()

id                   0
age                  0
workclass         1456
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1463
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     461
dtype: int64

In [53]:
X_test.isnull().sum()

id                  0
age                 0
workclass         380
fnlwgt              0
education           0
education.num       0
marital.status      0
occupation        380
relationship        0
race                0
sex                 0
capital.gain        0
capital.loss        0
hours.per.week      0
native.country    122
dtype: int64

In [54]:
X_train['workclass'].value_counts()

Private             18160
Self-emp-not-inc     2049
Local-gov            1648
State-gov            1037
Self-emp-inc          909
Federal-gov           770
Without-pay            12
Never-worked            7
Name: workclass, dtype: int64

In [55]:
X_train['occupation'].value_counts()

Exec-managerial      3323
Prof-specialty       3306
Craft-repair         3296
Adm-clerical         3037
Sales                2898
Other-service        2624
Machine-op-inspct    1584
Transport-moving     1257
Handlers-cleaners    1080
Farming-fishing       786
Tech-support          746
Protective-serv       521
Priv-house-serv       119
Armed-Forces            8
Name: occupation, dtype: int64

In [56]:
X_train['native.country'].value_counts()

United-States                 23381
Mexico                          516
Philippines                     158
Germany                         108
Canada                           88
Puerto-Rico                      87
El-Salvador                      76
India                            73
Cuba                             73
England                          69
Italy                            63
South                            62
Jamaica                          59
Vietnam                          57
China                            57
Guatemala                        54
Dominican-Republic               51
Japan                            49
Poland                           47
Columbia                         44
Taiwan                           37
Haiti                            37
Iran                             34
Portugal                         32
Peru                             29
Nicaragua                        27
Ecuador                          25
Greece                      

In [57]:
# 결측치는 최빈값과의 차이가 크면 최빈값으로
# 값이 비슷하면 별도의 값으로 대체

def data_fillna(df):
    df['workclass'] = df['workclass'].fillna(df['workclass'].mode()[0])
    df['occupation'] = df['occupation'].fillna("null")
    df['native.country'] = df["native.country"].fillna(df['native.country'].mode()[0])
    return df

X_train = data_fillna(X_train)
X_test = data_fillna(X_test)

X_train.isnull().sum()

id                0
age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
dtype: int64

### 데이터 전처리

In [58]:
# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

all_df = pd.concat([X_train.assign(ind='train'), X_test.assign(ind='test')])
le = LabelEncoder()
all_df[cat_features] = all_df[cat_features].apply(le.fit_transform)

X_train = all_df[all_df['ind'] == 'train']
X_train = X_train.drop('ind', axis=1)
X_train

X_test = all_df[all_df['ind'] == 'test']
X_test = X_test.drop('ind',axis=1)
X_test

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
20901,20901,58,3,114495,11,9,2,3,0,4,1,0,0,40,38
14170,14170,46,3,247043,11,9,2,13,0,4,1,0,0,40,38
1776,1776,67,1,103315,12,14,4,3,2,4,0,15831,0,72,38
30428,30428,18,3,165532,15,10,4,11,3,4,1,0,0,15,38
8602,8602,26,6,58039,15,10,2,7,0,4,1,0,0,40,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31222,31222,22,3,199426,15,10,4,14,1,4,0,0,0,40,38
10861,10861,41,3,155106,11,9,2,5,0,4,1,0,0,40,38
8929,8929,32,3,153078,9,13,4,7,1,1,1,0,0,40,34
2066,2066,48,6,171926,14,15,2,9,0,4,1,15024,0,50,38


In [59]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.fit_transform(X_test[numeric_features])

X_train

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
21851,21851,0.260274,3,0.156011,9,0.800000,2,2,0,4,1,0.000000,0.0,0.500000,38
7632,7632,0.493151,3,0.062255,12,0.866667,0,9,1,4,0,0.000000,0.0,0.397959,38
27878,27878,0.027397,3,0.129566,15,0.600000,4,12,1,4,0,0.000000,0.0,0.244898,38
14121,14121,0.041096,3,0.061343,11,0.533333,4,5,3,4,1,0.000000,0.0,0.295918,38
32345,32345,0.506849,6,0.085958,11,0.533333,2,9,0,4,1,0.000000,0.0,0.397959,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2669,2669,0.383562,3,0.118910,12,0.866667,0,3,4,4,1,0.074301,0.0,0.704082,38
17536,17536,0.260274,3,0.110039,1,0.400000,0,13,1,4,1,0.000000,0.0,0.397959,38
6201,6201,0.410959,3,0.178669,7,0.733333,6,7,3,4,0,0.000000,0.0,0.346939,38
27989,27989,0.452055,5,0.125113,10,1.000000,2,9,0,4,1,0.000000,0.0,0.224490,38


In [60]:
# 타겟 값 변경 !
y = (y_train['income'] != '<=50K').astype(int)
y[:5]


21851    1
7632     0
27878    0
14121    0
32345    0
Name: income, dtype: int64

### 검증용 데이터 분리

In [61]:
# 학습용 데이터와 검증용 데이터로 구분
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, test_size=0.15, random_state=2021)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((22140, 15), (3908, 15), (22140,), (3908,))

In [62]:
X_tr.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
1437,1437,0.191781,3,0.216501,9,0.800000,4,3,1,4,0,0.0,0.323232,0.397959,38
7151,7151,0.287671,5,0.127591,11,0.533333,2,2,0,4,1,0.0,0.000000,0.602041,38
30296,30296,0.424658,3,0.217452,9,0.800000,2,9,0,4,1,0.0,0.000000,0.346939,38
15372,15372,0.452055,3,0.142442,11,0.533333,2,2,0,4,1,0.0,0.000000,0.397959,38
13800,13800,0.178082,3,0.187243,15,0.600000,4,5,1,4,1,0.0,0.000000,0.397959,38


In [63]:
# 'id' 인덱스 삭제
X_tr = X_tr.drop('id', axis=1)
X_val = X_val.drop('id', axis=1)

X_tr.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
1437,0.191781,3,0.216501,9,0.800000,4,3,1,4,0,0.0,0.323232,0.397959,38
7151,0.287671,5,0.127591,11,0.533333,2,2,0,4,1,0.0,0.000000,0.602041,38
30296,0.424658,3,0.217452,9,0.800000,2,9,0,4,1,0.0,0.000000,0.346939,38
15372,0.452055,3,0.142442,11,0.533333,2,2,0,4,1,0.0,0.000000,0.397959,38
13800,0.178082,3,0.187243,15,0.600000,4,5,1,4,1,0.0,0.000000,0.397959,38


### 모델 생성 및 평가

In [64]:
# 의사 결정 나무
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier(random_state = 2023)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

accuracy_score(y_val, pred)

0.808085977482088

In [65]:
# 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state = 2023)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

accuracy_score(y_val, pred)

0.8503070624360286

In [67]:
X_test_id = X_test.pop('id')
pred = model.predict(X_test)

In [68]:
# 결과 csv 생성

output = pd.DataFrame({'id' : X_test_id, 'income':pred})
output.to_csv("2023111.csv", index=False)
output.head()

,id,income
20901,20901,1
14170,14170,0
1776,1776,1
30428,30428,0
8602,8602,0


In [69]:
# 채점 결과 (수험자는 확인 불가)
y_test = (y_test['income'] != '<=50K').astype(int)
from sklearn.metrics import accuracy_score
print('accuracy score:', (accuracy_score(y_test, pred)))

accuracy score: 0.8487640104406572
